# Yield calculation: drive ag model with climate model outputs

In [1]:
import numpy as np
import pandas as pd

### Regression coefficients

In [2]:
# Get coefficients
coeff = pd.read_csv('../fit_model/ag_model_coeff_final.csv')
coeff['GEOID'] = coeff['GEOID'].astype(str).str.zfill(5)

In [3]:
coeff.head()

,GEOID,GDDc,KDDc,Pc,P2c,R2,pval
0,01001,0.000650,-0.009133,0.638664,-0.306875,0.452173,7.862108e-05
1,01003,0.000000,-0.002219,1.515195,-0.644551,0.199122,7.038708e-02
2,01005,0.000148,-0.004832,0.820174,-0.524230,0.337459,2.971402e-03
3,01009,0.000977,-0.010829,0.000000,-0.538866,0.631368,7.569497e-08
4,01011,0.000290,-0.006119,0.000000,-0.099910,0.294675,2.883050e-02


## GMFD

In [4]:
# Import GMFD data
gmfd = pd.read_csv('../../agvars/GMFD/agvar_historical_gmfd.csv')
gmfd = gmfd.query('Year >= 2006 and Year <= 2016')
gmfd.sort_values(by = ['GEOID', 'Year'], inplace = True)
gmfd['GEOID'] = gmfd['GEOID'].astype(str).str.zfill(5)
gmfd['prcp2'] = gmfd['prcp']**2
gmfd.set_index(['GEOID', 'Year'], inplace = True)

In [5]:
# Subtract means
for agvar in gmfd.columns:
    gmfd[agvar] = gmfd[agvar] - gmfd[agvar].mean(level = 'GEOID')

In [6]:
gmfd

gdd       egdd      prcp     prcp2
GEOID Year                                           
01001 2006    4.726345  26.566847 -0.214764 -0.283854
      2007    8.849976  31.636496 -0.384405 -0.416301
      2008  -91.122431 -20.499031  0.102285  0.117987
      2009  -31.897592 -33.802521  0.224311  0.326226
      2010    6.120089  36.589416 -0.062858 -0.116409
...                ...        ...       ...       ...
56045 2012  234.291448  39.736328 -0.108562 -0.057078
      2013    9.251826 -13.109958 -0.038376 -0.024961
      2014 -104.457415 -27.874133  0.017893  0.007903
      2015  -25.506358 -21.060937  0.128836  0.091250
      2016   -8.042474  -6.686181 -0.021301 -0.015658

[34188 rows x 4 columns]

In [7]:
# Calculate Yields
gmfd = pd.merge(coeff, gmfd.reset_index(), on = ['GEOID'], how = 'inner')
gmfd.set_index(['GEOID', 'Year'], inplace = True)
gmfd['GMFD'] = gmfd['gdd']*gmfd['GDDc'] + gmfd['egdd']*gmfd['KDDc'] + gmfd['prcp']*gmfd['Pc'] + gmfd['prcp2']*gmfd['P2c']
gmfd = gmfd.filter(['GEOID', 'Year', 'GMFD'])

In [9]:
# Save
gmfd.to_csv('./output/GMFD/res_yield_06-16_gmfd.csv')

## NEX

In [10]:
nex_hist = ["agvar_historical_r1i1p1_ACCESS1-0.csv",
"agvar_historical_r1i1p1_BNU-ESM.csv",
"agvar_historical_r1i1p1_CCSM4.csv",
"agvar_historical_r1i1p1_CESM1-BGC.csv",
"agvar_historical_r1i1p1_CNRM-CM5.csv",
"agvar_historical_r1i1p1_CSIRO-Mk3-6-0.csv",
"agvar_historical_r1i1p1_CanESM2.csv",
"agvar_historical_r1i1p1_GFDL-CM3.csv",
"agvar_historical_r1i1p1_GFDL-ESM2G.csv",
"agvar_historical_r1i1p1_GFDL-ESM2M.csv",
"agvar_historical_r1i1p1_IPSL-CM5A-LR.csv",
"agvar_historical_r1i1p1_IPSL-CM5A-MR.csv",
"agvar_historical_r1i1p1_MIROC-ESM-CHEM.csv",
"agvar_historical_r1i1p1_MIROC-ESM.csv",
"agvar_historical_r1i1p1_MIROC5.csv",
"agvar_historical_r1i1p1_MPI-ESM-LR.csv",
"agvar_historical_r1i1p1_MPI-ESM-MR.csv",
"agvar_historical_r1i1p1_MRI-CGCM3.csv",
"agvar_historical_r1i1p1_NorESM1-M.csv",
"agvar_historical_r1i1p1_bcc-csm1-1.csv",
"agvar_historical_r1i1p1_inmcm4.csv"]

In [18]:
# Run calculation
def run_calc(name, hist=True, oos=False, mid=False, end=False):
    # Read in data
    data = pd.read_csv("../../agvars/NEX-GDDP/" + name)
    data["GEOID"] = data["GEOID"].astype(str).str.zfill(5)
    data.set_index(['GEOID', 'Year'], inplace=True)
    
    # Select time period
    if hist:
        data = data.query('Year >= 1960 and Year <= 2005')
    elif oos:
        data = data.query('Year >= 2006 and Year <= 2016')
    elif mid:
        data = data.query('Year >= 2040 and Year <= 2059')
    elif end:
        data = data.query('Year >= 2080 and Year <= 2099')
    
    # Square precip
    data['prcp2'] = data['prcp']**2
    
    # Subtract means
    for agvar in data.columns:
        data[agvar] = data[agvar] - data[agvar].mean(level = 'GEOID')

    # Calculate yields
    data = pd.merge(coeff, data.reset_index(), on = 'GEOID', how = 'outer')
    data['yield'] = data['gdd']*data['GDDc'] + data['egdd']*data['KDDc'] + data['prcp']*data['Pc'] + data['prcp2']*data['P2c']
    data = data.filter(['GEOID', 'Year', 'yield'])
    
    # Save
    timeID = hist*'60-05' + oos*'06-16' + mid*'40-59' + end*'80-99'
    data.to_csv("./output/NEX-GDDP/res_" + timeID + '_yield_' + name[6:], index=False)

In [19]:
for model in nex_hist:
    run_calc(model)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## CMIP

In [20]:
cmip_all = ["agvar_ACCESS1-0.historical+rcp85.csv",
"agvar_BNU-ESM.historical+rcp85.csv",
"agvar_CCSM4_historical+rcp85.csv",
"agvar_CESM1-BGC.historical+rcp85.csv",
"agvar_CNRM-CM5.historical+rcp85.csv",
"agvar_CSIRO-Mk3-6-0.historical+rcp85.csv",
"agvar_CanESM2.historical+rcp85.csv",
"agvar_GFDL-CM3.historical+rcp85.csv",
"agvar_GFDL-ESM2G.historical+rcp85.csv",
"agvar_GFDL-ESM2M.historical+rcp85.csv",
"agvar_IPSL-CM5A-LR.historical+rcp85.csv",
"agvar_IPSL-CM5A-MR.historical+rcp85.csv",
"agvar_MIROC-ESM-CHEM.historical+rcp85.csv",
"agvar_MIROC-ESM.historical+rcp85.csv",
"agvar_MIROC5.historical+rcp85.csv",
"agvar_MPI-ESM-LR.historical+rcp85.csv",
"agvar_MPI-ESM-MR.historical+rcp85.csv",
"agvar_MRI-CGCM3.historical+rcp85.csv",
"agvar_NorESM1-M.historical+rcp85.csv",
"agvar_bcc-csm1-1_historical+rcp85.csv",
"agvar_inmcm4.historical+rcp85.csv"]

In [21]:
# Run calculation
def run_calc(name, hist=True, oos=False, mid=False, end=False):
    # Read in data
    data = pd.read_csv("../../agvars/CMIP/" + name)
    data["GEOID"] = data["GEOID"].astype(str).str.zfill(5)
    data.set_index(['GEOID', 'Year'], inplace=True)
    
    # Select time period
    if hist:
        data = data.query('Year >= 1960 and Year <= 2005')
    elif oos:
        data = data.query('Year >= 2006 and Year <= 2016')
    elif mid:
        data = data.query('Year >= 2040 and Year <= 2059')
    elif end:
        data = data.query('Year >= 2080 and Year <= 2099')
    
    # Square precip
    data['prcp2'] = data['prcp']**2
    
    # Subtract means
    for agvar in data.columns:
        data[agvar] = data[agvar] - data[agvar].mean(level = 'GEOID')

    # Calculate yields
    data = pd.merge(coeff, data.reset_index(), on = 'GEOID', how = 'outer')
    data['yield'] = data['gdd']*data['GDDc'] + data['egdd']*data['KDDc'] + data['prcp']*data['Pc'] + data['prcp2']*data['P2c']
    data = data.filter(['GEOID', 'Year', 'yield'])
    
    # Save
    timeID = hist*'60-05' + oos*'06-16' + mid*'40-59' + end*'80-99'
    data.to_csv("./output/CMIP/res_" + timeID + '_yield_' + name[6:], index=False)

In [22]:
for model in cmip_all:
    run_calc(model)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
